In [ ]:
import numpy as np
import pandas as pd


: 

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/Colab_Dataset/MovieRecommonder/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/Colab_Dataset/MovieRecommonder/tmdb_5000_credits.csv')

: 

In [ ]:
movies.head(1)

: 

In [ ]:
credits.head(1)

: 

In [ ]:
movies = movies.merge(credits,on='title')

: 

In [ ]:
movies.head(1)

: 

In [ ]:
movies.info()

: 

In [ ]:
movies = movies[['movie_id', 'title', 'overview', 'genres','keywords','cast','crew']]

: 

In [ ]:
movies.head()

: 

In [ ]:
movies.isnull().sum()

: 

In [ ]:
movies.dropna(inplace=True)

: 

In [ ]:
movies.duplicated().sum()

: 

In [ ]:
movies.iloc[0].genres

: 

In [ ]:
import ast

: 

In [ ]:
def convert(obj):
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

: 

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

: 

In [ ]:
movies.head()

: 

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

: 

In [ ]:
movies.head()

: 

In [ ]:
def convert_cast(obj):
  L = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter != 3:
      L.append(i['name'])
      counter+=1
    else:
      break
  return L

: 

In [ ]:
movies['cast'] = movies['cast'].apply(convert_cast)

: 

In [ ]:
movies.head()

: 

In [ ]:
def fetch_director(obj):
  L = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
       L.append(i['name'])
       break
  return L

: 

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

: 

In [ ]:
movies.head()

: 

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

: 

In [ ]:
movies.head()

: 

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

: 

In [ ]:
movies.head()

: 

In [ ]:
movies['tags'] = movies['overview'] +  movies['keywords'] + movies['cast'] + movies['crew']

: 

In [ ]:
movies.head()

: 

In [ ]:
res = movies[['movie_id','title','tags']]

: 

In [ ]:
res['tags'] = res['tags'].apply(lambda x:" ".join(x))

: 

In [ ]:
res.head()

: 

In [ ]:
res['tags'] = res['tags'].apply(lambda x:x.lower())

: 

In [ ]:
res.head()

: 

In [ ]:
pip install nltk

: 

In [ ]:
import nltk

: 

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

: 

In [ ]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

: 

In [ ]:
res['tags'] = res['tags'].apply(stem)

: 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

: 

In [ ]:
vectors = cv.fit_transform(res['tags']).toarray()

: 

In [ ]:
vectors

: 

In [ ]:
cv.get_feature_names_out()

: 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

: 

In [ ]:
similarity = cosine_similarity(vectors)

: 

In [ ]:
similarity[1]

: 

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

: 

In [ ]:
res.iloc[1216].title

: 

In [ ]:
def recommend(movie):
    movie_index = res[res['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(res.iloc[i[0]].title)



: 

In [ ]:
recommend('Batman Begins')

: 

In [ ]:
import pickle

: 

In [ ]:

# Assuming 'res' is your model or data to be saved
pickle.dump(res.to_dict(), open('/content/drive/My Drive/movies_dict.pkl', 'wb'))

: 

In [ ]:
pickle.dump(similarity, open('/content/drive/My Drive/similarity.pkl', 'wb'))

: 